In [235]:
import numpy as np
import pandas as pd
import re
import datetime
from pandas import ExcelWriter

In [236]:
#read in data

members = pd.read_excel("data.xlsx",sheetname=2)
calls = pd.read_excel("data.xlsx",sheetname=3).dropna()
claims = pd.read_excel("data.xlsx",sheetname=4).rename(
    index=str,columns={"Member":"SubscriberID"})
costs = pd.read_excel("data.xlsx",sheetname=5)

In [237]:
def CullJunkDates(df,s):
    bools = []
    for val in s:
        if isinstance(val,datetime.datetime): 
            bools.append(True)
        else:
            bools.append(False)
            
    return df[bools]

In [238]:
claims['SubscriberID'] = claims['SubscriberID'].apply(lambda x: x.split("-")[0])

#standardize satisfaction rating

calls['Satisfaction'] = calls['Satisfaction'].map({
    'SAT':'Satisfied',
    r'\bD\b':'Dissatisfied',
    'VD':'Very Dissatisfied',
    'VS':'Very Satisfied',
    'Satisfied':'Satisfied',
    'Very Satisfied':'Very Satisfied',
    'Dissatisfied':'Dissatisfied',
    'Very Dissatisfied':'Very Dissatisfied'
})


        
#set timeframe to 2016
calls['CallYear'] = calls['CallDate'].apply(lambda x: x.year)
calls = calls[calls['CallYear'] == 2016]

claims = CullJunkDates(claims,claims['DateofService'])
claims['ClaimYear'] = claims['DateofService'].apply(lambda x: x.year)
claims = claims[claims['ClaimYear'] == 2016]

# add satisfaction index

calls['SatScore'] = calls['Satisfaction'].map( {
    'Very Dissatisfied':0,
    'Dissatisfied':1,
    'Satisfied':2,
    'Very Satisfied':3
})

calls.head(20)



,SubscriberID,CallDate,mon_num,Satisfaction,QSR_cd,CSP,Duration,CallYear,SatScore
0,150990125,2016-01-06 00:00:00,1,Satisfied,502.581441,yancey,33,2016,2.0
1,150029490,2016-03-16 00:00:00,3,Satisfied,469.192460,yancey,8,2016,2.0
2,150519251,2016-07-20 00:00:00,7,Satisfied,874.843469,yancey,1,2016,2.0
3,150644747,2016-06-19 00:00:00,6,Very Dissatisfied,800.301494,rojas,4,2016,0.0
4,150636919,2016-08-12 00:00:00,8,Satisfied,237.712789,nordquist,4,2016,2.0
5,150785388,2016-10-27 00:00:00,10,Very Satisfied,51.762178,hintz,38,2016,3.0
6,150867409,2016-01-02 00:00:00,1,Satisfied,229.353874,hintz,13,2016,2.0
7,150551525,2016-06-23 00:00:00,6,Satisfied,126.765904,hintz,7,2016,2.0
8,150262499,2016-05-31 00:00:00,5,Very Satisfied,848.062032,davishale,4,2016,3.0
9,150139103,2016-08-11 00:00:00,8,Very Dissatisfied,803.655557,hintz,16,2016,0.0


In [240]:
#export modified dfs back to excel

writer = ExcelWriter('cleandata.xlsx')
members.to_excel(writer,'members',index=False)
calls.to_excel(writer,'calls2016',index=False)
claims.to_excel(writer,'claims2016',index=False)
costs.to_excel(writer,'costs2016',index=False)
writer.save()